In [1]:
import pandas as pd
from nsepy import get_history as gh
import datetime as dt
import yfinance as yf

In [2]:
import datetime as dt

start = dt.datetime(2021, 7, 1)
end = dt.datetime(2022, 1, 27)
symbol = 'HDFCBANK.NS'  # Stock symbol in uppercase

stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [3]:
stk_data=stk_data[["Open","High","Low","Close"]]


In [4]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (143, 4)


In [5]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
dataset=data1

In [6]:

training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

114
X_train length: (114, 4)
X_test length: (29, 4)
y_train length: (114, 4)
y_test length: (29, 4)


In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}


In [9]:
def combination(dataset, listt):
    """
    Function to perform Simple Exponential Smoothing (SES) algorithm for forecasting.

    Parameters:
    - dataset: Pandas DataFrame or Series containing the time series data.
    - listt: List of variables to use for forecasting.

    Returns:
    - perf: DataFrame containing performance metrics.
    - model: Fitted SES model.
    - pred: Forecasted values.
    """
    print(listt)
    datasetTwo = dataset[listt]
    test_obs = 28
    train = datasetTwo[:-test_obs][listt[0]]  # Selecting the first variable for training
    test = datasetTwo[-test_obs:]

    from statsmodels.tsa.holtwinters import SimpleExpSmoothing
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

    performance = {
        "Model": [],
        "RMSE": [],
        "MaPe": [],
        "Test": []
    }

    best_alpha = None
    best_rmse = float('inf')

    for alpha in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:  # Try different alpha values
        model = SimpleExpSmoothing(train)
        fit_model = model.fit(smoothing_level=alpha)

        # Forecast
        pred = fit_model.forecast(len(test))  # Forecast for the same number of observations as test data

        # Calculate RMSE
        rmse = mean_squared_error(test[listt[0]], pred, squared=False)  # Selecting the first variable for comparison

        # Check if current model has lower RMSE
        if rmse < best_rmse:
            best_rmse = rmse
            best_alpha = alpha
            best_pred = pred

    print("Best alpha:", best_alpha)
    print("Best RMSE:", best_rmse)

    # Calculate MAPE
    mape = mean_absolute_percentage_error(test[listt[0]], best_pred)  # Selecting the first variable for comparison

    # Store performance metrics
    performance["Model"].append(listt)
    performance["RMSE"].append(best_rmse)
    performance["MaPe"].append(mape)
    performance["Test"].append(test_obs)

    perf = pd.DataFrame(performance)
    return perf, fit_model, best_pred


In [10]:
listt=["Close","High","Open","Low"]


In [11]:
perf,result,pred=combination(data1, listt)


['Close', 'High', 'Open', 'Low']
Best alpha: 0.8
Best RMSE: 0.16096424359177167


C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no free parameters to estimate. Set optimized=False to suppress this warning
  return func(*args, **kwargs)
C:\Users\kathi\anaconda3\envs\tsa\Lib\site-packages\pandas\util\_decorators.py:213: EstimationWarning: Model has no 

In [12]:
result

In [13]:
pred

115    0.305706
116    0.305706
117    0.305706
118    0.305706
119    0.305706
120    0.305706
121    0.305706
122    0.305706
123    0.305706
124    0.305706
125    0.305706
126    0.305706
127    0.305706
128    0.305706
129    0.305706
130    0.305706
131    0.305706
132    0.305706
133    0.305706
134    0.305706
135    0.305706
136    0.305706
137    0.305706
138    0.305706
139    0.305706
140    0.305706
141    0.305706
142    0.305706
dtype: float64

In [14]:
perf

,Model,RMSE,MaPe,Test
0,"[Close, High, Open, Low]",0.160964,1.012292,28
